# Modeling Exercises - Time Series

In [1]:
import pandas as pd
import numpy as np

# visualize 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# working with dates
from datetime import datetime

# to evaluated performance using rmse
from sklearn.metrics import mean_squared_error
from math import sqrt 

# for tsa 
import statsmodels.api as sm

# holt's linear trend model. 
from statsmodels.tsa.api import Holt

#custom module
import prepare as p
import acquire_func as a

### Exercises
The end result of this exercise should be a Jupyter notebook named `model`.

Using [saas.csv](https://ds.codeup.com/saas.csv) or log data from API usage or store_item_sales

1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum. 
2. Forecast, plot and evaluate using each of the 4 parametric based methods we discussed:
    - Simple Average
    - Moving Average
    - Holt's Linear Trend Model
    - Based on previous year/month/etc., this is up to you.

Optional: Using store item demand

1. Predict 2018 total **monthly** sales for a single store and/or item by creating a model using prophet.
2. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
3. Plot the 2018 monthly sales predictions.


----- 
SaaS - security as a service

Amount - is dollar amount

subscription_type - is the subscription type the customer has
    
    - see if subscription type is always the same dollar amount

In [3]:
df = pd.read_csv('saas.csv')

In [4]:
df.head()

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


- Looks like Month_invoiced is the date

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790458 entries, 0 to 790457
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Month_Invoiced     790458 non-null  object 
 1   Customer_Id        790458 non-null  int64  
 2   Invoice_Id         790458 non-null  int64  
 3   Subscription_Type  790458 non-null  float64
 4   Amount             790458 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 30.2+ MB


In [7]:
df.Month_Invoiced.value_counts().head(10)

2014-01-31    20000
2014-02-28    19800
2014-03-31    19599
2014-04-30    19413
2014-05-31    19221
2014-06-30    19038
2014-07-31    18851
2014-08-31    18672
2014-09-30    18494
2014-10-31    18313
Name: Month_Invoiced, dtype: int64

- Need to aggregate based on month
- sum up amount
- need to check subscription type

In [16]:
df.groupby(by = ['Subscription_Type', 'Amount']).count()

,,Month_Invoiced,Customer_Id,Invoice_Id
Subscription_Type,Amount,,,
0.0,0.0,496410,496410,496410
1.0,5.0,178212,178212,178212
2.0,10.0,70706,70706,70706
3.0,15.0,26474,26474,26474
4.0,20.0,18656,18656,18656


- looks like each plan is costs the same 

Plan = Amount

0.0 = 0.0

1.0 = 5.0

2.0 = 10.0

3.0 = 15.0

4.0 = 20.0

In [23]:
# change Month_invoiced to datetime
df['Month_Invoiced'] = pd.to_datetime(df.Month_Invoiced)

In [26]:
# reset index
df = df.set_index('Month_Invoiced')

In [38]:
# Resample on sum, select only the amount column and make sure it's a dataframe
df1 = df.resample('M').sum()['Amount'].to_frame()

In [40]:
df1.head()

,Amount
Month_Invoiced,
2014-01-31,35850.0
2014-02-28,37190.0
2014-03-31,38295.0
2014-04-30,39255.0
2014-05-31,40355.0
